In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time

In [2]:
xmin = -5.12
xmax = 5.12
dim = 10
pres = 3
longitud = int(math.log2( (xmax-xmin)*(10**pres) )+0.9)
prob_mut = 0.03
prob_cruza = 0.8
pobsize = 250
num_generaciones = 1000
num_padres = 16

dim_disc = dim * longitud

In [3]:
def bin2dec(bin_str):
    return int(''.join(str(x) for x in bin_str), 2)

In [4]:
def vbin2vdec(vbin, bits_per_var=longitud, num_vars=dim, vmin=xmin, vmax=xmax):
    vdec = []
    max_dec = 2**bits_per_var - 1
    for i in range(num_vars):
        bin_segment = vbin[i*bits_per_var:(i+1)*bits_per_var]
        dec = bin2dec(bin_segment)
        scaled_val = vmin + ((vmax - vmin) / max_dec) * dec
        vdec.append(scaled_val)
    return vdec

In [5]:
def aptitud(vbin):
    vdec = np.array(vbin2vdec(vbin))
    suma = np.sum(vdec**2 - 10 * np.cos(2 * np.pi * vdec))
    resultado = 10*dim + suma
    return resultado


def evaluar_aptitud(individuos):
    return [aptitud(ind) for ind in individuos]

In [6]:
def generate_pob(pobsize):
    pob = []
    for i in range(pobsize):        
        vector = np.random.randint(0, 2, size=dim_disc)
        pob.append(vector)
    return pob

In [7]:
def crossover_uniforme(v1, v2, prob_cruza):
    if np.random.rand() < prob_cruza:
        mascara = np.random.randint(0, 2, size=len(v1)).astype(bool)
        inversa_mascara = np.logical_not(mascara)
        h1 = np.where(mascara, v1, v2)
        h2 = np.where(mascara, v2, v1)
    else:
        h1 = v1
        h2 = v2
    return h1, h2

In [8]:
def mutacion_uniforme_binaria(vector, prob_mut):
    mascara = np.random.random(size=len(vector)) < prob_mut
    vector[mascara] = 1 - vector[mascara]    
    return vector

* Calcula la aptitud total de la población para determinar el tamaño de los "segmentos" de la ruleta para cada individuo.
* Establece un punto de inicio aleatorio y un intervalo constante (punto de ruleta) para realizar la selección.
* Recorre la población, acumulando la aptitud y seleccionando individuos cuando la aptitud acumulada supera los puntos establecidos por el intervalo.

In [9]:
def seleccion_universal_estocastica(poblacion, aptitudes, num_padres):
    max_aptitud = max(aptitudes)
    aptitudes_invertidas = [max_aptitud - a for a in aptitudes]
    aptitudes_sum = sum(aptitudes_invertidas)
    puntos_seleccion = np.linspace(0, aptitudes_sum, num_padres, endpoint=False)
    punto_inicio = np.random.uniform(0, aptitudes_sum / num_padres)
    puntos_seleccion += punto_inicio

    padres = []
    indice_actual = 0
    suma_acumulada = aptitudes_invertidas[indice_actual]
    for punto in puntos_seleccion:
        while suma_acumulada < punto:
            indice_actual += 1
            suma_acumulada += aptitudes_invertidas[indice_actual]
        padres.append(poblacion[indice_actual])
    return padres

In [10]:
# start_time = time.time()

# poblacion = generate_pob(pobsize)
# for gen in range(num_generaciones):
#     aptitudes = evaluar_aptitud(poblacion)
#     padres = seleccion_universal_estocastica(poblacion, aptitudes, num_padres)
    
#     nueva_poblacion = []
#     for i in range(0, num_padres, 2):
#         padre1 = padres[i]
#         padre2 = padres[i + 1]
#         hijo1, hijo2 = crossover_uniforme(padre1, padre2, prob_cruza)
#         hijo1 = mutacion_uniforme_binaria(hijo1, prob_mut)
#         hijo2 = mutacion_uniforme_binaria(hijo2, prob_mut)
#         nueva_poblacion.extend([hijo1, hijo2])
    
#     poblacion.extend(nueva_poblacion)
#     aptitudes = evaluar_aptitud(poblacion)
#     indices_mejores = np.argsort(aptitudes)[:pobsize]
#     poblacion = [poblacion[i] for i in indices_mejores]
#     print(f"generación {gen}, Mejor {aptitud(poblacion[0])}")
    
# end_time = time.time()
# elapsed_time = end_time - start_time

# print(f"El tiempo de ejecución es: {elapsed_time} segundos")

# # Resultados finales
# aptitudes_finales = evaluar_aptitud(poblacion)
# mejor_individuo = poblacion[np.argmin(aptitudes_finales)]
# mejor_aptitud = min(aptitudes_finales)

# print("Mejor aptitud:", mejor_aptitud)
# print("Mejor individuo:", mejor_individuo)
# print("Valores decodificados:", vbin2vdec(mejor_individuo))

In [ ]:

start_time = time.time()

poblacion = generate_pob(pobsize)
for gen in range(num_generaciones):
    aptitudes = evaluar_aptitud(poblacion)
    indices_mejores = np.argsort(aptitudes)[:5]
    elite = [poblacion[i] for i in indices_mejores]

    padres = seleccion_universal_estocastica(poblacion, aptitudes, num_padres)

    nueva_poblacion = elite[:]  # Agregar individuos élite a la nueva población
    while len(nueva_poblacion) < pobsize:
        for i in range(0, num_padres, 2):
            padre1 = padres[i]
            padre2 = padres[i + 1]
            hijo1, hijo2 = crossover_uniforme(padre1, padre2, prob_cruza)
            hijo1 = mutacion_uniforme_binaria(hijo1, prob_mut)
            hijo2 = mutacion_uniforme_binaria(hijo2, prob_mut)
            nueva_poblacion.extend([hijo1, hijo2])
            if len(nueva_poblacion) >= pobsize:
                break

    # Evaluar aptitudes de la nueva población expandida
    poblacion.extend(nueva_poblacion)
    poblacion = sorted(poblacion, key=aptitud)[:pobsize]  # Ordenar y reducir a pobsize

    # Guardar y mostrar el mejor individuo
    mejor_aptitud = aptitud(poblacion[0])

    # Guardar y mostrar el mejor individuo
    mejor_aptitud = min(evaluar_aptitud(poblacion))
    print(f"Generación {gen}, Mejor aptitud: {mejor_aptitud}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"El tiempo de ejecución es: {elapsed_time} segundos")

# Resultados finales
aptitudes_finales = evaluar_aptitud(poblacion)
mejor_individuo = poblacion[np.argmin(aptitudes_finales)]
mejor_aptitud = min(aptitudes_finales)

print("Mejor aptitud:", mejor_aptitud)
print("Mejor individuo:", mejor_individuo)
print("Valores decodificados:", vbin2vdec(mejor_individuo))

Generación 0, Mejor aptitud: 97.68715296502427
Generación 1, Mejor aptitud: 97.3577730198982
Generación 2, Mejor aptitud: 73.59634489553932
Generación 3, Mejor aptitud: 73.59634489553932
Generación 4, Mejor aptitud: 79.98198703740755
Generación 5, Mejor aptitud: 57.15287438759237
Generación 6, Mejor aptitud: 57.15287438759237
Generación 7, Mejor aptitud: 60.953875107979286
Generación 8, Mejor aptitud: 60.953875107979286
Generación 9, Mejor aptitud: 64.85353379894835
Generación 10, Mejor aptitud: 55.20082592671906
Generación 11, Mejor aptitud: 47.693054273914406
Generación 12, Mejor aptitud: 47.693054273914406
Generación 13, Mejor aptitud: 54.65365052037566
Generación 14, Mejor aptitud: 45.20657954131536
Generación 15, Mejor aptitud: 33.01363031454993
Generación 16, Mejor aptitud: 31.772019731457178
Generación 17, Mejor aptitud: 31.772019731457178
Generación 18, Mejor aptitud: 34.807772484547655
Generación 19, Mejor aptitud: 24.200417171350892
Generación 20, Mejor aptitud: 32.1466598538

Generación 168, Mejor aptitud: 18.683750958104426
Generación 169, Mejor aptitud: 15.745704036129354
Generación 170, Mejor aptitud: 16.27593258840858
Generación 171, Mejor aptitud: 17.078719276748757
Generación 172, Mejor aptitud: 18.551859606740436
Generación 173, Mejor aptitud: 18.551859606740436
Generación 174, Mejor aptitud: 18.551859606740436
Generación 175, Mejor aptitud: 19.21900030495219
Generación 176, Mejor aptitud: 18.619229102065006
Generación 177, Mejor aptitud: 18.619229102065006
Generación 178, Mejor aptitud: 18.619229102065006
Generación 179, Mejor aptitud: 18.619229102065006
Generación 180, Mejor aptitud: 21.412218784538354
Generación 181, Mejor aptitud: 16.6070272281044
Generación 182, Mejor aptitud: 16.6070272281044
Generación 183, Mejor aptitud: 16.6070272281044
Generación 184, Mejor aptitud: 21.505956598901747
Generación 185, Mejor aptitud: 17.701158134268027
Generación 186, Mejor aptitud: 17.701158134268027
Generación 187, Mejor aptitud: 18.597379746318808
Generaci

Generación 336, Mejor aptitud: 16.64924552026625
Generación 337, Mejor aptitud: 16.64924552026625
Generación 338, Mejor aptitud: 16.891306126415984
Generación 339, Mejor aptitud: 16.891306126415984
Generación 340, Mejor aptitud: 16.891306126415984
Generación 341, Mejor aptitud: 20.863423823023567
Generación 342, Mejor aptitud: 20.565196535562208
Generación 343, Mejor aptitud: 20.565196535562208
Generación 344, Mejor aptitud: 18.071687867923004
Generación 345, Mejor aptitud: 17.620341779453838
Generación 346, Mejor aptitud: 17.620341779453838
Generación 347, Mejor aptitud: 18.071687867923004
Generación 348, Mejor aptitud: 21.124986605162007
Generación 349, Mejor aptitud: 20.761578137286563
Generación 350, Mejor aptitud: 20.761578137286563
Generación 351, Mejor aptitud: 20.761578137286563
Generación 352, Mejor aptitud: 23.075089822053812
Generación 353, Mejor aptitud: 22.84263428068735
Generación 354, Mejor aptitud: 22.993747403138258
Generación 355, Mejor aptitud: 17.210586708392896
Gen

In [ ]:
# x=[]
# for i in range(30):
#     x.append(algoritmo_genetico(pobsize, num_generaciones, prob_mut, num_padres))
#     print(f"Se terminó de agregar la ejecucción [{i}]")

In [ ]:
# plt.figure(figsize=(10, 5))  
# plt.plot(x[1],  linestyle='-', color='b')
# plt.title('Gráfico de convergencia')  
# plt.xlabel('generación')  
# plt.ylabel('mejor aptitud')  
# plt.grid(True)

# plt.show()